In [2]:


import json

import sys
import time
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Load your IBM Quantum token from an environment variable named IBM_API_KEY
ibm_quantum_token = os.getenv("IBM_API_KEY")
if ibm_quantum_token is None:
    raise ValueError("IBM Quantum token not found. Please set the IBM_API_KEY environment variable.")

# Set the request URL and headers for job creation
reqUrl = "https://api.quantum-computing.ibm.com/runtime/jobs"
headersList = {
    "Accept": "application/json",
    "Authorization": f"Bearer {ibm_quantum_token}",
    "Content-Type": "application/json"
}

# Set the payload for job creation
payload = json.dumps({
    "program_id": "estimator",
    "backend": "ibm_brisbane",
    "hub": "ibm-q",
    "group": "open",
    "project": "main",
    "params": {
        "pubs": [[
            "OPENQASM 3.0; include \"stdgates.inc\"; bit[1] c; x $0; c[0] = measure $0;", "Z"
        ]],
        "options": {"dynamical_decoupling": {"enable": True}},
        "version": 2,
        "resilience_level": 1
    }
})

# Function to create a job
def create_job():
    response = requests.post(reqUrl, headers=headersList, data=payload)
    if response.status_code == 200:
        response_json = response.json()
        job_id = response_json.get('id')
        print(f"Job ID: {job_id}")
        return job_id
    else:
        print(f"Failed to create job. Status code: {response.status_code}, Response: {response.text}")
        return None

# Function to check job status
def check_job_status(job_id):
    status_url = f"https://api.quantum-computing.ibm.com/runtime/jobs/{job_id}"
    response = requests.get(status_url, headers=headersList)
    if response.status_code == 200:
        response_json = response.json()
        status = response_json.get('status')
        print(f"Job Status: {status}")
        return status
    else:
        print(f"Failed to check job status. Status code: {response.status_code}, Response: {response.text}")
        return None

# Function to retrieve job results
def retrieve_job_results(job_id):
    result_url = f"https://api.quantum-computing.ibm.com/runtime/jobs/{job_id}/result"
    response = requests.get(result_url, headers=headersList)
    if response.status_code == 200:
        print("Job Results:")
        print(response.json())
    else:
        print(f"Failed to retrieve job results. Status code: {response.status_code}, Response: {response.text}")

# Main function to create and monitor the job
def main():
    job_id = create_job()
    if job_id:
        while True:
            status = check_job_status(job_id)
            if status and status not in ['Pending', 'Running']:
                break
            time.sleep(30)  # Wait for 30 seconds before checking again

        if status == 'Completed':
            retrieve_job_results(job_id)
        else:
            print(f"Job did not complete successfully. Final status: {status}")

if __name__ == "__main__":
    main()

Job ID: cv2t8njhdzz00084qdv0
Job Status: Running
Job Status: Failed
Job did not complete successfully. Final status: Failed


In [5]:

import json
import numpy as np
import sys
import time
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Load your IBM Quantum token from an environment variable named IBM_API_KEY
ibm_quantum_token = os.getenv("IBM_API_KEY")
if ibm_quantum_token is None:
    raise ValueError("IBM Quantum token not found. Please set the IBM_API_KEY environment variable.")

# Set the request URL and headers for job creation
reqUrl = "https://api.quantum-computing.ibm.com/runtime/jobs"
headersList = {
    "Accept": "application/json",
    "Authorization": f"Bearer {ibm_quantum_token}",
    "Content-Type": "application/json"
}

# Set the payload for job creation
payload = json.dumps({
    "program_id": "estimator",
    "backend": "ibm_brisbane",
    "hub": "ibm-q",
    "group": "open",
    "project": "main",
    "params": {
        "circuits": [
            {
                "name": "example",
                "quantum_circuit": "OPENQASM 3.0; include \"stdgates.inc\"; bit[1] c; x $0; c[0] = measure $0;"
            }
        ],
        "meas_level": 2,
        "shots": 1024
    }
})

# Function to create a job
def create_job():
    try:
        response = requests.post(reqUrl, headers=headersList, data=payload)
        response.raise_for_status()
        response_json = response.json()
        job_id = response_json.get('id')
        print(f"Job ID: {job_id}")
        return job_id
    except requests.exceptions.RequestException as e:
        print(f"Failed to create job. Error: {e}")
        return None

# Function to check job status
def check_job_status(job_id):
    status_url = f"https://api.quantum-computing.ibm.com/runtime/jobs/{job_id}"
    try:
        response = requests.get(status_url, headers=headersList)
        response.raise_for_status()
        response_json = response.json()
        status = response_json.get('status')
        print(f"Job Status: {status}")
        return status
    except requests.exceptions.RequestException as e:
        print(f"Failed to check job status. Error: {e}")
        return None

# Function to retrieve job results
def retrieve_job_results(job_id):
    result_url = f"https://api.quantum-computing.ibm.com/runtime/jobs/{job_id}/result"
    try:
        response = requests.get(result_url, headers=headersList)
        response.raise_for_status()
        print("Job Results:")
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve job results. Error: {e}")

# Main function to create and monitor the job
def main():
    job_id = create_job()
    if job_id:
        while True:
            status = check_job_status(job_id)
            if status and status not in ['Pending', 'Running']:
                break
            time.sleep(30)  # Wait for 30 seconds before checking again

        if status == 'Completed':
            retrieve_job_results(job_id)
        else:
            print(f"Job did not complete successfully. Final status: {status}")

if __name__ == "__main__":
    main()

Job ID: cv2v6bhx55b0008j0hx0
Job Status: Queued
Job did not complete successfully. Final status: Queued


In [7]:
 
# Optional: List all the instances you can access.
service = QiskitRuntimeService(channel='ibm_quantum')
print(service.instances())
 
# Optional: Specify it at service level. This becomes the default unless overwritten.
service = QiskitRuntimeService(channel='ibm_quantum', instance="hub1/group1/project1")
backend1 = service.backend("ibmq_manila")
 
# Optional: Specify it at the backend level, which overwrites the service-level specification when this backend is used.
backend2 = service.backend("ibmq_manila", instance="hub2/group2/project2")
 
sampler1 = SamplerV2(mode=backend1)    # this will use hub1/group1/project1
sampler2 = SamplerV2(mode=backend2)    # this will use hub2/group2/project2

AccountNotFoundError: 'Unable to find account.'